In [78]:
import pandas as pd
import numpy as np
import os
import shutil
import plotly.express as px
import plotly.graph_objects as go
import importlib
import performance_shr
importlib.reload(performance_shr)
from performance_shr import convert_to_float

In [79]:
file_dir = 'D:/Astro/Images/2024-12-12/NGC 3372/LIGHTS'
file_name = 'ImageMetaData.csv'
file_path = os.path.join(file_dir, file_name)
converters_dict = {
    'FilePath': str,   
    'Gain': convert_to_float,   
    'DetectedStars': convert_to_float,   
    'HFR': convert_to_float,  
    'HFRStDev': convert_to_float,  
    'FWHM': convert_to_float,  
    'Eccentricity': convert_to_float,  
    'FocuserPosition': convert_to_float,  
    'Airmass': convert_to_float,  
    'MountRA': convert_to_float,  
    'MountDec': convert_to_float,  
    }

In [80]:
data = pd.read_csv(file_path, converters=converters_dict)
data['FileName'] = data['FilePath'].apply(lambda x: os.path.basename(x))

In [81]:
def filter_percentiles(df, col='HFR', top=0.05, bottom=0.05, zero=True):
    top_threshold = df[col].quantile(1 - top) if top else False
    bottom_threshold = df[col].quantile(bottom) if bottom else False
    is_na = df[col].isna()
    is_false = is_na != is_na
    is_zero = df[col] == 0 if zero else is_false
    is_top = df[col] >= top_threshold if top_threshold else is_false
    is_bottom = df[col] <= bottom_threshold if bottom_threshold else is_false
    is_filtered = is_zero | is_na | is_top | is_bottom
    print(f'{col}: TopThreshold={top_threshold:.3f} n({is_top.sum()}), BottomThreshold={bottom_threshold:.3f} n({is_bottom.sum()}), na=n({is_na.sum()}), zero=n({is_zero.sum()}), filtered=n({is_filtered.sum()})')
    return is_filtered

In [88]:
print(f'**** Rejecting Images based on top and bottom percentiles for HFR, FWHM, Eccentricity and #Stars Dectected ****')
is_badHFR = filter_percentiles(data, col='HFR', bottom=False)
is_badHFRStDev = filter_percentiles(data, col='HFRStDev', bottom=False)
is_badFWHM = filter_percentiles(data, col='FWHM', bottom=False)
is_badEccentricity = filter_percentiles(data, col='Eccentricity')
is_badDetectedStars = filter_percentiles(data, col='DetectedStars', top=False)
is_rejected = is_badFWHM | is_badHFR | is_badFWHM | is_badHFRStDev | is_badEccentricity | is_badDetectedStars
print(f'Total Rejected: n({is_rejected.sum()})')


**** Rejecting Images based on top and bottom percentiles for HFR, FWHM, Eccentricity and #Stars Dectected ****
HFR: TopThreshold=2.907 n(37), BottomThreshold=0.000 n(0), na=n(0), zero=n(14), filtered=n(51)
HFRStDev: TopThreshold=0.276 n(37), BottomThreshold=0.000 n(0), na=n(0), zero=n(14), filtered=n(51)
FWHM: TopThreshold=6.990 n(36), BottomThreshold=0.000 n(0), na=n(32), zero=n(0), filtered=n(68)
Eccentricity: TopThreshold=0.705 n(36), BottomThreshold=0.262 n(36), na=n(32), zero=n(0), filtered=n(104)
DetectedStars: TopThreshold=0.000 n(0), BottomThreshold=53.000 n(38), na=n(0), zero=n(11), filtered=n(38)
Total Rejected: n(133)


In [89]:
# Create the 'rejected' directory at the same level as file_dir if it doesn't exist 
parent_dir = os.path.dirname(file_dir) 
rejected_dir = os.path.join(parent_dir, 'Rejected') 
os.makedirs(rejected_dir, exist_ok=True)

In [90]:
rejects = data[is_rejected].FileName
for filename in rejects:
    src_path = os.path.join(file_dir, filename)
    dest_path = os.path.join(rejected_dir, filename)
    if os.path.exists(src_path): 
        shutil.move(src_path, dest_path) 
        print(f'Moved: {filename}')

print('File moving complete.')

File moving complete.
